In [20]:
import numpy as np
from ortools.graph.pywrapgraph import SimpleMinCostFlow

In [98]:
# Create graph
# 1 start and 1 stop. All intercenected via some nodes

seed = 1
n_int = int(1e7) # N intconecting_nodes

edges = np.concatenate([
    np.stack([0*np.ones(n_int), np.arange(2, n_int+2)], axis=1),
    np.stack([np.arange(2, n_int+2), 1*np.ones(n_int)], axis=1)
]).astype('int32')

costs = np.random.randint(low=0, high=100, size=len(edges)).astype('int32')
capacities = np.random.randint(low=1, high=n_int, size=len(edges)).astype('int32')

supplies = np.concatenate([[1, -1], np.zeros(n_int)]).astype('int32')

N_edges = edges.shape[0]
N_nodes = len(supplies)

# Current interface

In [99]:
min_cost_flow = SimpleMinCostFlow()

In [100]:
%%time
for i in range(0, N_edges):
    min_cost_flow.AddArcWithCapacityAndUnitCost(int(edges[i, 0]), int(edges[i, 1]),
                                                int(capacities[i]), int(costs[i]))

CPU times: user 1min 15s, sys: 2.72 s, total: 1min 18s
Wall time: 1min 30s


In [101]:
%%time
for i in range(0, N_nodes):
    min_cost_flow.SetNodeSupply(i, int(supplies[i]))

CPU times: user 16.2 s, sys: 84 ms, total: 16.3 s
Wall time: 18.2 s


In [102]:
%%time
if min_cost_flow.Solve() != min_cost_flow.OPTIMAL:
    raise Exception('There was an issue with the min cost flow input.')

CPU times: user 16.6 s, sys: 3.89 s, total: 20.5 s
Wall time: 23.3 s


In [103]:
%%time
flow = np.array([min_cost_flow.Flow(i) for i in range(N_edges)])

CPU times: user 19.7 s, sys: 504 ms, total: 20.2 s
Wall time: 23.4 s


# Cython interface

In [104]:
min_cost_flow_vec = SimpleMinCostFlowVectorized()

In [105]:
%%time
min_cost_flow_vec.AddArcWithCapacityAndUnitCostVectorized(edges[:,0], edges[:,1], capacities, costs)

CPU times: user 11.3 s, sys: 664 ms, total: 12 s
Wall time: 12.4 s


In [106]:
%%time
min_cost_flow_vec.SetNodeSupplyVectorized(np.arange(N_nodes, dtype='int32'), supplies)

CPU times: user 3.37 s, sys: 12 ms, total: 3.38 s
Wall time: 3.39 s


In [107]:
%%time
if min_cost_flow_vec.Solve() != min_cost_flow_vec.OPTIMAL:
    raise Exception('There was an issue with the min cost flow input.')

CPU times: user 15.2 s, sys: 3.63 s, total: 18.9 s
Wall time: 20 s


In [108]:
%%time
flow = min_cost_flow_vec.FlowVectorized(np.arange(N_edges, dtype='int32'))

CPU times: user 8.31 s, sys: 296 ms, total: 8.61 s
Wall time: 9.07 s


| Step          | Current interface | Cython interface |
|---------------|-------------------|------------------|
| AddArc        | 90 s              | 12.4 s           |
| SetNodeSupply | 18.2 s            | 3.39 ms          |
| Solve         | 23.3 s            | 20 s             |
| Flow          | 23.4 s            | 9.07 s           |
| **Total**         | **154.9 s**           | **41.1 s**          |

# Time comparision

# Cython interface code

In [76]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

from ortools.graph._pywrapgraph import \
    SimpleMinCostFlow_AddArcWithCapacityAndUnitCost,\
    SimpleMinCostFlow_SetNodeSupply,\
    SimpleMinCostFlow_Flow

DTYPE = np.int32
ctypedef np.int32_t DTYPE_t


@cython.boundscheck(False)
@cython.wraparound(False)
def SimpleMinCostFlow_AddArcWithCapacityAndUnitCostVectorized(
        self,
        np.ndarray[DTYPE_t, ndim=1] tail,
        np.ndarray[DTYPE_t, ndim=1] head,
        np.ndarray[DTYPE_t, ndim=1] capacity,
        np.ndarray[DTYPE_t, ndim=1] unit_cost):

    cdef int len = tail.shape[0]

    assert tail.dtype == DTYPE
    assert head.dtype == DTYPE
    assert capacity.dtype == DTYPE
    assert unit_cost.dtype == DTYPE
    assert head.shape[0] == len
    assert capacity.shape[0] == len
    assert unit_cost.shape[0] == len

    for i in range(len):
        SimpleMinCostFlow_AddArcWithCapacityAndUnitCost(self, tail[i], head[i], capacity[i], unit_cost[i])


@cython.boundscheck(False)
@cython.wraparound(False)
def SimpleMinCostFlow_SetNodeSupplyVectorized(self,
                                              np.ndarray[DTYPE_t, ndim=1] node,
                                              np.ndarray[DTYPE_t, ndim=1] supply):
    cdef int len = node.shape[0]

    assert node.dtype == DTYPE
    assert supply.dtype == DTYPE
    assert supply.shape[0] == len

    for i in range(len):
        SimpleMinCostFlow_SetNodeSupply(self, node[i], supply[i])


@cython.boundscheck(False)
@cython.wraparound(False)
def SimpleMinCostFlow_FlowVectorized(self,
                                     np.ndarray[DTYPE_t, ndim=1] arc):

    cdef int len = arc.shape[0]

    assert arc.dtype == DTYPE

    cdef np.ndarray flow = np.zeros(len, dtype=DTYPE)

    for i in range(len):
        flow[i] = SimpleMinCostFlow_Flow(self, arc[i])

    return flow

In [77]:
class SimpleMinCostFlowVectorized(SimpleMinCostFlow):

    def AddArcWithCapacityAndUnitCostVectorized(self, tail, head, capacity, unit_cost):
        return SimpleMinCostFlow_AddArcWithCapacityAndUnitCostVectorized(self, tail, head, capacity, unit_cost)

    def SetNodeSupplyVectorized(self, node, supply):
        return SimpleMinCostFlow_SetNodeSupplyVectorized(self, node, supply)

    def FlowVectorized(self, arc):
        return SimpleMinCostFlow_FlowVectorized(self, arc)